# Train Multi-GPU single machine

Run on V100 8x in tests.

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re, json
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import dask
import time
print(tf.__version__)

2.4.1


In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: skirmer (use `wandb login --relogin` to force relogin)


True

In [3]:

# USE categorical_crossentropy

#     # Data
#     tester_flow = image.ImageDataGenerator()
#     train_ds = tester_flow.flow_from_directory(
#         directory = '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/train',
#         target_size=(224,224),
#         batch_size=batch_size_dist
#     )

#     valid_ds = tester_flow.flow_from_directory(
#         directory = '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/validation',
#         target_size=(224,224),
#         batch_size=batch_size_dist
#     )

#     # Data
#     train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#       '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/train',
#       seed=123,
#       image_size=(224,224),
#       batch_size=batchsize).prefetch(5)
#     # USE sparse_categorical_crossentropy

#     valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
#       '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/validation',
#       seed=123,
#       image_size=(224,224),
#       batch_size=batchsize).prefetch(5)

In [4]:
def train_multigpu(n_epochs, base_lr, batchsize,wbargs, scale_batch = False, scale_lr = False):
    
    wbargs={**wbargs, "scale_batch":scale_batch, "scale_lr":scale_lr}

    # --------- Start wandb --------- #
    wandb.init(entity='skirmer', project='tensorflow_mg_tests',
    config=wbargs)

    strategy = tf.distribute.MirroredStrategy()
    print('Number of devices: %d' % strategy.num_replicas_in_sync)
    if scale_batch:
        batchsize = batchsize*strategy.num_replicas_in_sync
        
    if scale_lr:
        base_lr = base_lr*strategy.num_replicas_in_sync
    
    with strategy.scope():
        model = tf.keras.applications.ResNet50(
            include_top=True,
            weights=None,
            classes=10)

        optimizer = keras.optimizers.Adam(lr=base_lr)
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #integer labels

    # Data
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/train',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2)
        
    valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/validation',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2)
    
    start = time.time()

    model.fit(
        train_ds, 
        epochs=n_epochs, 
        validation_data=valid_ds,
        callbacks=[WandbCallback()]
    )
    end = time.time()-start
    print("model training time", end)
    wandb.log({"training_time":end})

    # Close your wandb run 
    wandb.run.finish()
    
    tf.keras.models.save_model(model, 'model/keras_multi/')

In [5]:
model_params = {'n_epochs': 50, 
                'base_lr': .02,
               'batchsize': 32}

wbargs = {**model_params,
    'classes':10,
    'Notes':"tf_v100_8x",
    'Tags': ['multi', 'gpu', 'tensorflow'],
    'dataset':"Clothing",
    'architecture':"ResNet50"}

In [6]:

tester_plain = train_multigpu(wbargs=wbargs, **model_params)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:loc

epoch,49
loss,0.03656
accuracy,0.9899
val_loss,1.52194
val_accuracy,0.70674
_runtime,531
_timestamp,1629395502
_step,50
best_val_loss,0.97786
best_epoch,28
training_time,521.33897


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▁▁▁▁▁▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇▇▇███▇█████████
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▃▃▃▂▃▃▄▄▄▅▄▆▅▇▆▆▇▇▇▇▇█▇█▇▇█▇▇██▇▇▇▇█▇█
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training_time,▁


INFO:tensorflow:Assets written to: model/keras_multi/assets


In [7]:
tester_batchscale = train_multigpu(scale_batch = True, wbargs=wbargs, **model_params)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
Found 3068 files belonging to 10 classes.
Found 341 files belonging to 10 classes.
Epoch 1/50
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
24/24 [==============================] - 83s 519ms/step - loss: 11.9749 - accuracy: 0.1446 - val_loss: 81737052294077218816.0000 - val_accuracy: 0.0733
Epoch 2/50
24/24 [==============================] - 9s 286ms/step - loss: 2.4048 - accuracy: 0.2562 - val_loss: 9091143434240.0000 - val_accuracy: 0.0938
Epoch 3/50
24/24 [==============================] - 10s 311ms/step - loss: 2.1457 - accuracy: 0.2548 - val_loss: 1185608064.0000 - val_accuracy: 0.0938
Epoch 4/

epoch,49
loss,0.23286
accuracy,0.92047
val_loss,2.22049
val_accuracy,0.55718
_runtime,579
_timestamp,1629396115
_step,50
best_val_loss,1.39135
best_epoch,39
training_time,572.81465


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
accuracy,▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇█████
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▂▂▃▃▃▄▄▄▄▄▅▄▄▆▅▅▄▄▅▃▅▇▇▆▆▇▅▇██▆█▇
_runtime,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training_time,▁


INFO:tensorflow:Assets written to: model/keras_multi/assets


In [8]:
# tester_lrscale = train_multigpu(scale_lr = True, wbargs=wbargs, **model_params)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
Found 3068 files belonging to 10 classes.
Found 341 files belonging to 10 classes.
Epoch 1/50
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
96/96 [==============================] - 87s 149ms/step - loss: 16.7420 - accuracy: 0.2072 - val_loss: 7323102.0000 - val_accuracy: 0.1437
Epoch 2/50
96/96 [==============================] - 8s 78ms/step - loss: 2.1255 - accuracy: 0.2592 - val_loss: 2580.8357 - val_accuracy: 0.1848
Epoch 3/50
96/96 [==============================] - 8s 78ms/step - loss: 2.1198 - accuracy: 0.2593 - val_loss: 290.7189 - val_accuracy: 0.1994
Epoch 4/50
96/96 [======================

epoch,49
loss,2.13425
accuracy,0.25619
val_loss,143.43649
val_accuracy,0.23754
_runtime,517
_timestamp,1629396667
_step,50
best_val_loss,2.17783
best_epoch,46
training_time,510.7649


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▆
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▇█▇█▆███████████████████████████▃████
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training_time,▁


INFO:tensorflow:Assets written to: model/keras_multi/assets


In [9]:
tester_both = train_multigpu(scale_lr = True, scale_batch = True, wbargs=wbargs, **model_params)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
Found 3068 files belonging to 10 classes.
Found 341 files belonging to 10 classes.
Epoch 1/50
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
24/24 [==============================] - 87s 554ms/step - loss: 21.1472 - accuracy: 0.1686 - val_loss: 17286139428138410573824.0000 - val_accuracy: 0.0938
Epoch 2/50
24/24 [==============================] - 9s 284ms/step - loss: 2.4909 - accuracy: 0.2614 - val_loss: 30472354660352.0000 - val_accuracy: 0.0411
Epoch 3/50
24/24 [==============================] - 9s 282ms/step - loss: 2.1271 - accuracy: 0.2674 - val_loss: 2418614272.0000 - val_accuracy: 0.0411
Epoch

epoch,49
loss,0.39854
accuracy,0.86571
val_loss,2.46663
val_accuracy,0.55425
_runtime,572
_timestamp,1629397273
_step,50
best_val_loss,1.75031
best_epoch,37
training_time,565.41998


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇██
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▂▁▁▁▁▁▂▃▃▃▄▄▅▄▅▅▄▅▄▄▅▄▄▄▆▅▆▆▇▇█▇▆▇▇▇▆▆▇█
_runtime,▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training_time,▁


INFO:tensorflow:Assets written to: model/keras_multi/assets
